In [1]:
# Import necessary modules
import sys
sys.path.append("..")
from src.confpickem.yahoo_pickem_scraper import YahooPickEm, calculate_player_stats
from src.confpickem.confidence_pickem_sim import ConfidencePickEmSimulator, Player
from src.confpickem.yahoo_pickem_integration import convert_yahoo_to_simulator_format, convert_yahoo_picks_to_dataframe
import pandas as pd

In [2]:
# Pull data from Yahoo
week = 1
league_id = 15435
num_sims = 1000
# num_entries = 50

yahoo_data = YahooPickEm(week, league_id, cookies_file='../src/confpickem/cookies.txt')
yahoo_data.games
# yahoo_data.games
# yahoo_data.players
# yahoo_data.results

,favorite,favorite_pick_pct,underdog,underdog_pick_pct,home_favorite,favorite_confidence,underdog_confidence,spread,win_prob,kickoff_time
0,Phi,97.0,Dal,3.0,True,13.2,4.4,8.5,0.7635,2025-09-04 20:20:00-05:00
1,KC,84.0,LAC,16.0,False,9.4,3.8,3.0,0.5930,2025-09-05 20:00:00-05:00
2,TB,84.0,Atl,16.0,False,8.0,4.8,2.5,0.5775,2025-09-07 13:00:00-05:00
3,Cin,95.0,Cle,5.0,False,11.3,4.7,5.5,0.6705,2025-09-07 13:00:00-05:00
4,Ind,27.0,Mia,73.0,True,4.5,6.4,1.0,0.5310,2025-09-07 13:00:00-05:00
5,NE,69.0,LV,31.0,True,5.8,4.7,2.5,0.5775,2025-09-07 13:00:00-05:00
6,Ari,93.0,NO,7.0,False,10.2,5.5,6.5,0.7015,2025-09-07 13:00:00-05:00
7,Pit,90.0,NYJ,10.0,False,9.2,4.5,3.0,0.5930,2025-09-07 13:00:00-05:00
8,Was,96.0,NYG,4.0,True,11.8,5.0,6.0,0.6860,2025-09-07 13:00:00-05:00
9,Jax,89.0,Car,11.0,True,8.2,4.6,3.5,0.6085,2025-09-07 13:00:00-05:00


In [3]:
# Initialize simulator instance and feed in NFL game details
simulator = ConfidencePickEmSimulator(num_sims=num_sims)
games_df = convert_yahoo_to_simulator_format(yahoo_data, ignore_results=False)#True)
simulator.add_games_from_dataframe(games_df)

# games_df
simulator.games

Known results for 2 out of 16 games.
Results dict: {'Phi': 'Phi', 'KC': 'LAC'}


[Game(home_team='Phi', away_team='Dal', vegas_win_prob=0.7635000000000001, crowd_home_pick_pct=0.97, crowd_home_confidence=13.2, crowd_away_confidence=4.4, week=1, kickoff_time=Timestamp('2025-09-04 20:20:00-0500', tz='UTC-05:00'), actual_outcome=True, picks_locked=False),
 Game(home_team='LAC', away_team='KC', vegas_win_prob=0.40700000000000003, crowd_home_pick_pct=0.16, crowd_home_confidence=3.8, crowd_away_confidence=9.4, week=1, kickoff_time=Timestamp('2025-09-05 20:00:00-0500', tz='UTC-05:00'), actual_outcome=True, picks_locked=False),
 Game(home_team='Atl', away_team='TB', vegas_win_prob=0.4225, crowd_home_pick_pct=0.16, crowd_home_confidence=4.8, crowd_away_confidence=8.0, week=1, kickoff_time=Timestamp('2025-09-07 13:00:00-0500', tz='UTC-05:00'), actual_outcome=None, picks_locked=False),
 Game(home_team='Cle', away_team='Cin', vegas_win_prob=0.3295, crowd_home_pick_pct=0.05, crowd_home_confidence=4.7, crowd_away_confidence=11.3, week=1, kickoff_time=Timestamp('2025-09-07 13:00:

In [4]:
# # Create average players and feed into simulator
# simulator.players = [
#         Player(
#             name=f"Player {ind + 1}" if ind > 0 else "Firman's Educated Guesses",
#             skill_level=0.749,
#             crowd_following=0.737,
#             confidence_following=0.826
#         ) for ind in range(50)
#     ]

# Calculate player statistics from previous results
stats = calculate_player_stats(
        league_id=league_id,
        weeks=range(1, week + 1), # CHANGE THIS BACK!!!
        cookies_file='../src/confpickem/cookies.txt'
    )
stats['me'] = stats.player == "Firman's Educated Guesses"
stats = stats.sort_values(by="me",ascending=False,ignore_index=True)

# player_stats = [
#         Player(
#             name=stats.iloc[ind].player,
#             skill_level=stats.iloc[ind].skill_level,
#             crowd_following=stats.iloc[ind].crowd_following,
#             confidence_following=stats.iloc[ind].confidence_following
#         ) for ind in range(stats.shape[0])
#     ]
player_stats = [
        Player(
            name=stats.iloc[ind].player,
            skill_level=0.749,
            crowd_following=0.737,
            confidence_following=0.826
        ) for ind in range(stats.shape[0])
    ]
simulator.players = player_stats

simulator.players

[Player(name="Firman's Educated Guesses", skill_level=0.749, crowd_following=0.737, confidence_following=0.826),
 Player(name='ShowMeYourTDs', skill_level=0.749, crowd_following=0.737, confidence_following=0.826),
 Player(name='Tone', skill_level=0.749, crowd_following=0.737, confidence_following=0.826),
 Player(name='Bucky Swirving', skill_level=0.749, crowd_following=0.737, confidence_following=0.826),
 Player(name='Back to Back Buckeye Natty', skill_level=0.749, crowd_following=0.737, confidence_following=0.826),
 Player(name='Brock', skill_level=0.749, crowd_following=0.737, confidence_following=0.826),
 Player(name='Pay up $ucka!', skill_level=0.749, crowd_following=0.737, confidence_following=0.826),
 Player(name='mk3415', skill_level=0.749, crowd_following=0.737, confidence_following=0.826),
 Player(name='Rcdinero', skill_level=0.749, crowd_following=0.737, confidence_following=0.826),
 Player(name='Fast Eddie', skill_level=0.749, crowd_following=0.737, confidence_following=0.82

In [5]:
# dir(yahoo_data)
# yahoo_data.players

# Pull fixed picks so far from Yahoo player dataframe
fixed_picks = {}
num_games = max([int(col.split("_")[1]) for col in yahoo_data.players.columns if col.startswith('game_')])
for ind in range(yahoo_data.players.shape[0]):
    player_data = yahoo_data.players.iloc[ind]
    for game in range(1, num_games + 1):
        game_col = f"game_{game}_pick"
        conf_col = f"game_{game}_confidence"
        if pd.notna(player_data[game_col]) and pd.notna(player_data[conf_col]):
            if player_data.player_name not in fixed_picks:
                fixed_picks[player_data.player_name] = {}
            fixed_picks[player_data.player_name][player_data[game_col]] = int(player_data[conf_col])

# # Correction for myself (don't enter picks early next time...)
# fixed_picks["Firman's Educated Guesses"] = {'Phi': 16}

fixed_picks


{'ShowMeYourTDs': {'Phi': 16, 'LAC': 5},
 'Mskrlac': {'Phi': 16, 'LAC': 2},
 'dbalz': {'Phi': 16, 'LAC': 1},
 'MainGangJefe': {'Phi': 16, 'LAC': 1},
 "TD's Nutz": {'Phi': 16, 'KC': 10},
 '#FlyTheW': {'Phi': 16, 'KC': 3},
 'Ben’s Bears': {'Phi': 16, 'KC': 15},
 'Go Pack Go': {'Phi': 16, 'KC': 8},
 'Big oski no.1': {'Phi': 16, 'KC': 15},
 'Do You Fell Lucky Punk': {'Phi': 15, 'LAC': 1},
 'VT': {'Phi': 16, 'KC': 9},
 'More Birdies': {'Phi': 16, 'KC': 2},
 'Gman': {'Phi': 16, 'KC': 14},
 'Boats N Hoes': {'Phi': 16, 'KC': 15},
 'Dardanes': {'Phi': 16, 'KC': 6},
 'Fishy Picks': {'Phi': 16, 'KC': 7},
 "Firman's Educated Guesses": {'Phi': 16, 'KC': 7},
 'Mopar': {'Phi': 16, 'KC': 4},
 'RICHDOG !!!!!!': {'Phi': 16, 'KC': 15},
 'Rent’s Due Again': {'Phi': 16, 'KC': 7},
 'Loveland for ROY': {'Phi': 16, 'KC': 2},
 'Comeback Szn': {'Phi': 16, 'KC': 12},
 'Lenny': {'Phi': 15, 'KC': 14},
 'OB1': {'Phi': 12, 'LAC': 3},
 'Johnsonmaia06': {'Phi': 15, 'KC': 10},
 'Jason B': {'Phi': 15, 'KC': 14},
 'BIZAR

In [6]:

# Run simulation with random picks and outcomes
picks_df = simulator.simulate_picks(fixed_picks=fixed_picks)
outcomes = simulator.simulate_outcomes()
# picks_df.groupby(["player","game","picked_home"]).size().sort_values(ascending=False)
# outcomes.sum(axis=0)

# Analyze results
stats_orig = simulator.analyze_results(picks_df, outcomes)
for stat in stats_orig:
    stats_orig[stat] = stats_orig[stat].to_frame(stat).reset_index().rename(columns={"index":"player"})

summary_orig = pd.merge(left=stats_orig["win_pct"],right=stats_orig["expected_points"],how="inner",on="player")
summary_orig = pd.merge(left=summary_orig,right=stats_orig["point_std"],how="inner",on="player")
summary_orig = pd.merge(left=summary_orig,right=stats_orig["value_at_risk"],how="inner",on="player")
summary_orig.sort_values(by="win_pct",ascending=False)

,player,win_pct,expected_points,point_std,value_at_risk
48,ShowMeYourTDs,0.036,88.906,19.080546,56.95
19,Do You Fell Lucky Punk,0.028,85.773,19.284381,54.00
39,Mskrlac,0.027,85.481,18.810462,55.00
42,Pay up $ucka!,0.026,84.431,19.710448,52.00
0,#FlyTheW,0.025,84.649,18.451836,54.00
37,Mopar,0.025,83.609,19.526428,51.00
52,VT,0.024,84.191,18.940255,54.00
50,TD's Nutz,0.024,85.109,19.003164,54.00
11,Big oski no.1,0.024,82.908,19.579911,51.95
54,mk3415,0.023,84.621,19.143036,54.00


In [ ]:
# Identify optimal picks for the week
optimal_picks = simulator.optimize_picks("Firman's Educated Guesses",fixed_picks=fixed_picks,confidence_range=4)

fixed_picks["Firman's Educated Guesses"] = optimal_picks
print(optimal_picks)

Starting simulated annealing optimization for Firman's Educated Guesses
Initial solution value: 0.0020
Iteration 0: Current=0.0020, Best=0.0020, Temp=2.0000
Optimization complete. Best value: 0.0020
{'Phi': 16, 'KC': 7, 'TB': 12, 'Cle': 8, 'Mia': 4, 'LV': 9, 'NO': 11, 'Pit': 13, 'Was': 15, 'Car': 2, 'Den': 10, 'Sea': 5, 'Det': 3, 'LAR': 14, 'Buf': 1, 'Min': 6}


In [ ]:
print(fixed_picks)

In [ ]:
# # Setting my picks to the optimized values
# mask = (picks_df.player == simulator.players[0].name)
# for team, pts in optimal_picks.items():
#     game_mask = picks_df[mask].game.str.contains(team)
#     is_home = (picks_df.loc[mask & game_mask, 'game'].str.split('@').str[1] == team).astype(float)
#     picks_df.loc[mask & game_mask, 'picked_home'] = is_home
#     picks_df.loc[mask & game_mask, 'confidence'] = float(pts)

# Run simulation with optimized picks for my team
picks_df = simulator.simulate_picks(fixed_picks)
outcomes = simulator.simulate_outcomes()

# Analyze results
stats_new = simulator.analyze_results(picks_df, outcomes)
for stat in stats_new:
    stats_new[stat] = stats_new[stat].to_frame(stat).reset_index().rename(columns={"index":"player"})
summary_new = pd.merge(left=stats_new["win_pct"],right=stats_new["expected_points"],how="inner",on="player")
summary_new = pd.merge(left=summary_new,right=stats_new["point_std"],how="inner",on="player")
summary_new = pd.merge(left=summary_new,right=stats_new["value_at_risk"],how="inner",on="player")
summary_new.sort_values(by="win_pct",ascending=False)


In [ ]:
# Pulling actual picks from Yahoo
# picks_df_actual = convert_yahoo_picks_to_dataframe(yahoo_data, num_sims)#, optimal_picks)
picks_df_actual = convert_yahoo_picks_to_dataframe(yahoo_data, num_sims)#, fixed_picks)
print(picks_df_actual.loc[picks_df_actual.player == "Firman's Educated Guesses",["game","picked_home"]].drop_duplicates())
# picks_df.picked_home.unique()

# Simulating win probabilities given actual picks
outcomes = simulator.simulate_outcomes()
stats_actual = simulator.analyze_results(picks_df_actual, outcomes)
for stat in stats_actual:
    stats_actual[stat] = stats_actual[stat].to_frame(stat).reset_index().rename(columns={"index":"player"})
summary_actual = pd.merge(left=stats_actual["win_pct"],right=stats_actual["expected_points"],how="inner",on="player")
summary_actual = pd.merge(left=summary_actual,right=stats_actual["point_std"],how="inner",on="player")
summary_actual = pd.merge(left=summary_actual,right=stats_actual["value_at_risk"],how="inner",on="player")
summary_actual.sort_values(by="win_pct",ascending=False)

In [ ]:
# Identifying relative importance of each game
# importance = simulator.assess_game_importance("Firman's Educated Guesses", picks_df_actual)
importance = simulator.assess_game_importance("Firman's Educated Guesses", picks_df)
print("\nGame Importance Analysis:")
print(importance[["game","points_bid","win_probability","loss_probability","total_impact"]].sort_values('total_impact', ascending=False).to_string(index=False))

In [ ]:
# # Checking who actually won
# simulator = ConfidencePickEmSimulator(num_sims=num_sims)
# games_df = convert_yahoo_to_simulator_format(yahoo_data, ignore_results=False)
# simulator.add_games_from_dataframe(games_df)
# outcomes = simulator.simulate_outcomes()

# stats_actual = simulator.analyze_results(picks_df_actual, outcomes)
# for stat in stats_actual:
#     stats_actual[stat] = stats_actual[stat].to_frame(stat).reset_index().rename(columns={"index":"player"})
# summary_actual = pd.merge(left=stats_actual["win_pct"],right=stats_actual["expected_points"],how="inner",on="player")
# summary_actual = pd.merge(left=summary_actual,right=stats_actual["point_std"],how="inner",on="player")
# summary_actual = pd.merge(left=summary_actual,right=stats_actual["value_at_risk"],how="inner",on="player")
# summary_actual.sort_values(by=["win_pct","expected_points"],ascending=False)

In [ ]:
outcomes.sum(axis=0)